In [8]:
import sys
sys.path.append('../keys')  

from config import ALPHA_VANTAGE_API_KEY
import requests
import pandas as pd

In [5]:
def getData(ticker, ALPHA_VANTAGE_API_KEY):
    # Pulling from two parts of the API
    # See Documentation Here: https://www.alphavantage.co/documentation/
    income_statement_url = f'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={ticker}&apikey={ALPHA_VANTAGE_API_KEY}'
    balance_sheet_url = f'https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol={ticker}&apikey={ALPHA_VANTAGE_API_KEY}'
    cash_flow_url = f'https://www.alphavantage.co/query?function=CASH_FLOW&symbol={ticker}&apikey={ALPHA_VANTAGE_API_KEY}'

    try:
        # Fetching from multiple sources:
        # Income statement
        income_statement_response = requests.get(income_statement_url)
        income_statement_data = income_statement_response.json()
        
        # Balance sheet
        balance_sheet_response = requests.get(balance_sheet_url)
        balance_sheet_data = balance_sheet_response.json()

        # Cash_flow
        cash_flow_response = requests.get(cash_flow_url)
        cash_flow_data = cash_flow_response.json()

        # Check if data is valid before creating DataFrames
        if 'annualReports' in income_statement_data and 'annualReports' in balance_sheet_data and 'annualReports' in cash_flow_data:
            income_statement_df = pd.DataFrame(income_statement_data['annualReports'])
            balance_sheet_df = pd.DataFrame(balance_sheet_data['annualReports'])
            cash_flow_df = pd.DataFrame(cash_flow_data['annualReports'])

            return income_statement_df, balance_sheet_df, cash_flow_df
        else:
            raise ValueError("Invalid data format")
    except (KeyError, IndexError, requests.exceptions.RequestException, ValueError) as e:
        print(f"Error fetching financial data: {e}")
        return None, None, None

ticker = 'WFC'
income_statement_df, balance_sheet_df, cash_flow_df = getData(ticker, ALPHA_VANTAGE_API_KEY)

if income_statement_df is not None:
    print("Income Statement DataFrame:")
    print(income_statement_df.head())  
    
    print("\nBalance Sheet DataFrame:")
    print(balance_sheet_df.head())
    
    print("\nCash Flow DataFrame:")
    print(cash_flow_df.head())
else:
    print("Failed to fetch financial data.")

Income Statement DataFrame:
  fiscalDateEnding reportedCurrency  grossProfit  totalRevenue costOfRevenue  \
0       2023-12-31              USD  18675000000  115340000000   61658000000   
1       2022-12-31              USD  15943000000   82859000000   59813000000   
2       2021-12-31              USD  36523000000   82407000000   50768000000   
3       2020-12-31              USD  -8597000000   82227000000   73067000000   
4       2019-12-31              USD  23924000000   85063000000   61139000000   

  costofGoodsAndServicesSold operatingIncome sellingGeneralAndAdministrative  \
0                  697000000      8594000000                      3696000000   
1                  750000000      6164000000                      3386000000   
2                  867000000     25756000000                      3568000000   
3                 1000000000    -22065000000                      3863000000   
4                 1200000000    -15953000000                     32148000000   

  research

In [10]:
income_statement_df.head()

,fiscalDateEnding,reportedCurrency,grossProfit,totalRevenue,costOfRevenue,costofGoodsAndServicesSold,operatingIncome,sellingGeneralAndAdministrative,researchAndDevelopment,operatingExpenses,...,depreciation,depreciationAndAmortization,incomeBeforeTax,incomeTaxExpense,interestAndDebtExpense,netIncomeFromContinuingOperations,comprehensiveIncomeNetOfTax,ebit,ebitda,netIncome
0,2023-12-31,USD,18675000000,115340000000,61658000000,697000000,8594000000,3696000000,None,10081000000,...,None,1300000000,21749000000,2607000000,32743000000,19029000000,20924000000,54492000000,55792000000,19142000000
1,2022-12-31,USD,15943000000,82859000000,59813000000,750000000,6164000000,3386000000,None,9779000000,...,None,1200000000,15269000000,2251000000,9074000000,12882000000,2061000000,24343000000,25543000000,13677000000
2,2021-12-31,USD,36523000000,82407000000,50768000000,867000000,25756000000,3568000000,None,10693000000,...,None,1400000000,27126000000,5578000000,3915000000,23238000000,19652000000,31041000000,32441000000,21548000000
3,2020-12-31,USD,-8597000000,82227000000,73067000000,1000000000,-22065000000,3863000000,None,13397000000,...,None,1400000000,2220000000,-1157000000,7963000000,3586000000,4882000000,10183000000,11583000000,3377000000
4,2019-12-31,USD,23924000000,85063000000,61139000000,1200000000,-15953000000,32148000000,None,35955000000,...,None,1400000000,25476000000,5761000000,18852000000,20041000000,24093000000,44328000000,45728000000,19715000000


In [11]:
num_cols = len(balance_sheet_df.columns) - 2
cols = list(balance_sheet_df.columns[2:]) 
balance_sheet_transformed_df = balance_sheet_df[cols]
balance_sheet_transformed_df = balance_sheet_transformed_df.apply(pd.to_numeric, errors='coerce', axis=1)

num_cols = len(income_statement_df.columns) - 2
cols = list(income_statement_df.columns[2:]) 
income_statement_transformed_df = income_statement_df[cols]
income_statement_transformed_df = income_statement_transformed_df.apply(pd.to_numeric, errors='coerce', axis=1)

num_cols = len(cash_flow_df.columns) - 2
cols = list(cash_flow_df.columns[2:]) 
cash_flow_transformed_df = cash_flow_df[cols]
cash_flow_transformed_df = cash_flow_transformed_df.apply(pd.to_numeric, errors='coerce', axis=1)

In [12]:
# Extracting needed variables

# Current ratio = totalCurrentAssets / totalCurrentLiabilities
    # Reference: https://www.investopedia.com/terms/c/currentratio.asp 
totalCurrentAssets = balance_sheet_transformed_df['totalCurrentAssets'].iloc[0]
totalCurrentLiabilities = balance_sheet_transformed_df['totalCurrentLiabilities'].iloc[0]
currentRatio = totalCurrentAssets / totalCurrentLiabilities

# Long-term Debt / Capital = longTermDebt / (longTermDebt + totalShareholderEquity)
    # Reference: https://www.investopedia.com/terms/l/longtermdebt-capitalization.asp *Preferred stock doesn't appear to be available on Alpha Vantage and isn't always issued
longTermDebt = balance_sheet_transformed_df['longTermDebt'].iloc[0]
totalShareholderEquity = balance_sheet_transformed_df['totalShareholderEquity'].iloc[0]
longTermDebtCapital = longTermDebt / (longTermDebt + totalShareholderEquity)

# Debt/Equity Ratio = totalLiabilities / totalShareholderEquity
    # Reference: https://www.investopedia.com/terms/d/debtequityratio.asp
totalLiabilities = balance_sheet_transformed_df['totalLiabilities'].iloc[0]
totalShareholderEquity = balance_sheet_transformed_df['totalShareholderEquity'].iloc[0]
debtEquityRatio= totalLiabilities / totalShareholderEquity

# Gross Margin = 100 × (totalRevenue - costofGoodsAndServicesSold) / totalRevenue
    # Reference: https://www.investopedia.com/terms/g/grossmargin.asp, https://www.omnicalculator.com/finance/margin#gross-margin-formula
totalRevenue = income_statement_transformed_df['totalRevenue'].iloc[0]
costofGoodsAndServicesSold = income_statement_transformed_df['costofGoodsAndServicesSold'].iloc[0]
grossMargin = ((totalRevenue - costofGoodsAndServicesSold) / totalRevenue) * 100

# Operating Margin = operatingIncome / totalRevenue
    # Reference: https://www.investopedia.com/terms/o/operatingmargin.asp
operatingIncome = income_statement_transformed_df['operatingIncome'].iloc[0]
operatingMargin = operatingIncome / totalRevenue

# EBIT Margin: ((totalRevenue - costofGoodsAndServicesSold - operatingExpenses) / totalRevenue) * 100
    # Reference: https://www.investopedia.com/terms/e/ebit.asp
operatingExpenses = income_statement_transformed_df['operatingExpenses'].iloc[0]
ebitMargin = ((totalRevenue - costofGoodsAndServicesSold - operatingExpenses) / totalRevenue) * 100

# EBITDA Margin: (incomeBeforeTax + depreciationAndAmortization) / totalRevenue
    # Reference: https://www.investopedia.com/terms/e/ebitda-margin.asp
incomeBeforeTax = income_statement_transformed_df['incomeBeforeTax'].iloc[0]
depreciationAndAmortization = income_statement_transformed_df['depreciationAndAmortization'].iloc[0]
ebitdaMargin = (incomeBeforeTax + depreciationAndAmortization) / totalRevenue

# Pre-Tax Profit Margin: (incomeBeforeTax / totalRevenue) * 100
    # Reference: https://www.investopedia.com/terms/p/pretax-margin.asp
preTaxProfitMargin = (incomeBeforeTax / totalRevenue) * 100

# Net Profit Margin: (netIncome / totalRevenue) * 100
    # Reference: https://www.investopedia.com/terms/n/net_margin.asp
netIncome = income_statement_transformed_df['netIncome'].iloc[0]
netProfitMargin = (netIncome / totalRevenue) * 100

# Asset Turnover Ratio: totalRevenue / ((totalAssets + totalAssetsPrevious) / 2)
    # totalAssetsPrevious: totalAssets(from preYear) [1] = previous year
totalAssets = balance_sheet_transformed_df['totalAssets'].iloc[0]
totalAssetsPrevious = balance_sheet_transformed_df['totalAssets'].iloc[1]
assetTurnoverRatio = totalRevenue / ((totalAssets + totalAssetsPrevious) / 2)

# ROE - Return On Equity: netIncome / ((totalShareholderEquity(curYear) + totalShareholderEquity(preYear)) / 2)
    # Reference: https://www.investopedia.com/terms/r/returnonequity.asp
totalShareholderEquityPrevious = balance_sheet_transformed_df['totalShareholderEquity'].iloc[1]
returnOnEquity = netIncome / ((totalShareholderEquity + totalShareholderEquityPrevious) / 2)

# Return On Tangible Equity: netIncome / (avgShareholderEquity - intangibleAssets)
    # Reference: https://www.wallstreetprep.com/knowledge/return-on-tangible-equity-rote/
totalShareholderEquity = balance_sheet_transformed_df['totalShareholderEquity'].iloc[0]
totalShareholderEquityPrevious = balance_sheet_transformed_df['totalShareholderEquity'].iloc[1]
avgShareholderEquity = (totalShareholderEquity + totalShareholderEquityPrevious) / 2
intangibleAssets = balance_sheet_transformed_df['intangibleAssets'].iloc[0]
returnOnTangibleEquity = netIncome / (avgShareholderEquity - intangibleAssets)

# ROA - Return On Assets: netIncome / totalAssets
    # Reference: https://www.investopedia.com/terms/r/returnonassets.asp
returnOnAssets = netIncome / totalAssets

# ROI - Return On Investment: (netIncome / ((totalShareholderEquity(curYear) + totalShareholderEquity(preYear)) / 2)) * 100 
    # Reference: https://www.wallstreetprep.com/knowledge/return-on-equity-roe/, https://www.investopedia.com/terms/r/returnoninvestment.asp
returnOnInvestment = (netIncome / ((totalShareholderEquity + totalShareholderEquityPrevious) / 2)) * 100

# Operating Cash Flow Per Share: operatingCashflow / commonStockSharesOutstanding
    # Reference: https://www.investopedia.com/terms/c/cashflowpershare.asp, https://www.wallstreetprep.com/knowledge/cash-flow-per-share/
operatingCashflow = cash_flow_transformed_df['operatingCashflow'].iloc[0]
commonStockSharesOutstanding = balance_sheet_transformed_df['commonStockSharesOutstanding'].iloc[0]
operatingCashFlowPerShare = operatingCashflow / commonStockSharesOutstanding

# Free Cash Flow Per Share: (operatingCashflow - capitalExpenditures) / commonStockSharesOutstanding
    # Reference: https://www.investopedia.com/terms/f/freecashflowpershare.asp
capitalExpenditures = cash_flow_transformed_df['capitalExpenditures'].iloc[0]
freeCashFlowPerShare = (operatingCashflow - capitalExpenditures) / commonStockSharesOutstanding



        # Create a DataFrame to store the results
metrics_df = pd.DataFrame({
    "Current Ratio": [currentRatio],
    "Long-term Debt Capital": [longTermDebtCapital],
    "Debt-Equity Ratio": [debtEquityRatio],
    "Gross Margin": [grossMargin],
    "Operating Margin": [operatingMargin],
    "EBIT Margin": [ebitMargin],
    "EBITDA Margin": [ebitdaMargin],
    "Pre-Tax Profit Margin": [preTaxProfitMargin],
    "Net Profit Margin": [netProfitMargin],
    "Asset Turnover Ratio": [assetTurnoverRatio],
    "Return On Equity": [returnOnEquity],
    "Return On Tangible Equity": [returnOnTangibleEquity],
    "Return On Assets": [returnOnAssets],
    "Return On Investment": [returnOnInvestment],
    "Operating Cash Flow Per Share": [operatingCashFlowPerShare],
    "Free Cash Flow Per Share": [freeCashFlowPerShare]
})
metrics_df.head()

,Current Ratio,Long-term Debt Capital,Debt-Equity Ratio,Gross Margin,Operating Margin,EBIT Margin,EBITDA Margin,Pre-Tax Profit Margin,Net Profit Margin,Asset Turnover Ratio,Return On Equity,Return On Tangible Equity,Return On Assets,Return On Investment,Operating Cash Flow Per Share,Free Cash Flow Per Share
0,3.790317,0.52778,9.395241,99.3957,0.07451,90.655453,0.199835,18.856424,16.596151,0.060491,0.104612,0.121301,0.009905,10.461195,11.147079,NaN
